In [ ]:
make_corpus(subreddit_object)

clean_all_docs(docs)

get_subreddit(subreddit_name, app_name, oa_script, oa_secret, pwd)

get_cos_sim(model, word_1, word_2)

compare_cos_sim(model, word_list, comp_word)

